In [2]:
import re
import importlib
import inspect

from pathlib import Path


In [3]:

PYPROJECT = Path.cwd().joinpath("../pyproject.toml").resolve(strict=True)
ROOTDIR = PYPROJECT.parent
SRCDIR = ROOTDIR / "src"

In [4]:
#config = toml.load(PYPROJECT)
#config

In [5]:
def list_modules(srcdir):
    srcdir = Path(srcdir)
    modules = []
    for path in srcdir.rglob("*.py"):
        path = path.relative_to(SRCDIR)
        name = path.as_posix().removesuffix(".py").replace("/", ".")
        name = name.removesuffix(".__init__")
        modules.append(name)
    return modules

list_modules(SRCDIR)

['mintalib.functions',
 'mintalib.reflib',
 'mintalib',
 'mintalib.indicators',
 'mintalib.model',
 'mintalib.utils',
 'mintalib.testing',
 'mintalib.samples']

In [6]:
from mintalib.functions import EMA, DEMA

def trim_doc(doc):
    doc = re.sub("^ *\n", "", doc)
    doc = re.sub("\n+$", "\n", doc)
    doc = re.sub(" +$", "", doc, re.MULTILINE)

    indents = list(re.findall(r"^ *(?=\S)", doc, flags=re.MULTILINE))

    if indents and indents[0] == "":
        indents = indents[1:]

    if indents:
        dedent = min(indents)
        doc = re.sub(f"^{dedent}", "", doc, flags=re.MULTILINE)

    return doc


def convert_doc(doc):
    doc = trim_doc(doc)
    doc = re.sub(
        r"^[ ]+ (?= \w+ [ ]* [(:=] .+)", "- ", doc,
        flags = re.MULTILINE | re.VERBOSE)
    doc = re.sub(
        r"(?<= [:\n]\n) [ ]+", "+ ", doc,
        flags = re.MULTILINE | re.VERBOSE)
    return doc

print(EMA.__doc__)
doc = convert_doc(EMA.__doc__)
print(doc)


    Exponential Moving Average

    Args:
        period (int) : time period, required
        adjust (bool) : whether to adjust weights, default False
            when true update ratio increases gradually (see formula)

    Formula:
        EMA is calculated as a recursive formula
        The standard formula is ema += alpha * (value - ema)
            with alpha = 2.0 / (period + 1.0)
        The adjusted formula is ema = num/div
            where num = value + rho * num, div = 1.0 + rho * div
            with rho = 1.0 - alpha

    Attributes:
        same_scale = True

Exponential Moving Average

Args:
- period (int) : time period, required
- adjust (bool) : whether to adjust weights, default False
        when true update ratio increases gradually (see formula)

Formula:
+ EMA is calculated as a recursive formula
    The standard formula is ema += alpha * (value - ema)
        with alpha = 2.0 / (period + 1.0)
    The adjusted formula is ema = num/div
        where num = value +

In [7]:
from collections import namedtuple

def get_ftype(func):
    if isinstance(func, type):
        return "class"

    if callable(func):
        if func.__module__.endswith("indicators"):
            return "indicator"
        return "function"

    return None

def get_signature(func):
    result = str(inspect.signature(func))
    result = re.sub("(?<=: )'(\w+)'", r"\1", result)
    return result 

def make_doc(module_name, func_names=None, *, strict: bool=False):
    output = ""

    module = importlib.import_module(module_name, package=None)

    output += f"# {module_name} module\n\n"

    if module.__doc__:
        doc = trim_doc(module.__doc__)
        output += doc + "\n\n"

    if func_names is None:
        func_names = [n for n in dir(module) if n.isupper()]
    elif callable(func_names):
        condition = func_names
        func_names = [n for n in dir(module) if condition(n)]

    for name in  func_names:
        func = getattr(module, name)
        ftype = get_ftype(func)
        if not ftype:
            continue
        if strict and func.__module__ != module_name:
            continue
        signature = get_signature(func)
        output += f"## {name} {ftype}\n\n"
        output += f"```python\n{name}{signature}\n```\n\n"
        if func.__doc__:
            doc = convert_doc(func.__doc__)
            output += doc
        output += "\n"

    return output

output = make_doc("mintalib.functions")
print (output)


# mintalib.functions module

Mintalib Functions

Function names are upper case.

Functions that accept a prices dataframe input have a first paramater called `prices`.
Functions that accept a series input have a fist parameter called `series`,
and an optional parameter `item` to specify which column to use on dataframe inputs.

All functions wrap their output to match the type of their input.
In particular the result of a function applied to a pandas series or dataframes
will have the same index as the input.


## ADX function

```python
ADX(prices, period: int = 14)
```

Average Directional Index

Args:
- period (int) : time period, default 14

## ATR function

```python
ATR(prices, period: int = 14)
```

Average True Range

Args:
- period (int) : time period, default 14    

## AVGPRICE function

```python
AVGPRICE(prices)
```

Average Price

Value of (open + high + low + close) / 4

Attributes:
- same_scale = True

## BBANDS function

```python
BBANDS(prices, period: int = 20, nbdev

In [8]:
def save_doc(module, func_names=None, *, strict: bool = False):
    docs = ROOTDIR / "docs"
    docs.mkdir(parents=True, exist_ok=True)

    output = make_doc(
        module,
        func_names=func_names,
        strict=strict
    )

    print(output)

    mdfile = docs / f"{module}.md"
    mdfile.write_text(output)

save_doc("mintalib.functions")

# mintalib.functions module

Mintalib Functions

Function names are upper case.

Functions that accept a prices dataframe input have a first paramater called `prices`.
Functions that accept a series input have a fist parameter called `series`,
and an optional parameter `item` to specify which column to use on dataframe inputs.

All functions wrap their output to match the type of their input.
In particular the result of a function applied to a pandas series or dataframes
will have the same index as the input.


## ADX function

```python
ADX(prices, period: int = 14)
```

Average Directional Index

Args:
- period (int) : time period, default 14

## ATR function

```python
ATR(prices, period: int = 14)
```

Average True Range

Args:
- period (int) : time period, default 14    

## AVGPRICE function

```python
AVGPRICE(prices)
```

Average Price

Value of (open + high + low + close) / 4

Attributes:
- same_scale = True

## BBANDS function

```python
BBANDS(prices, period: int = 20, nbdev

In [9]:

save_doc("mintalib.indicators")

# mintalib.indicators module

Mintalib Indicators

Indicators names are upper case.

Indicators offer a composable interface where a function is bound with its calculation parameters.
When instantiated with parameters an indicator yields a callable that can be applied to prices or series data.
Indicators support the `@` operator as syntactic sugar to apply the indicator to data.
So for example `SMA(50) @ prices` can be used to compute the 50 period simple moving average on `prices`,
insted of `SMA(50)(prices)`.


## ADX indicator

```python
ADX(period: int = 14)
```

Average Directional Index

Args:
- period (int) : time period, default 14

## ATR indicator

```python
ATR(period: int = 14)
```

Average True Range

Args:
- period (int) : time period, default 14    

## AVGPRICE indicator

```python
AVGPRICE()
```

Average Price

Value of (open + high + low + close) / 4

Attributes:
- same_scale = True

## BBANDS indicator

```python
BBANDS(period: int = 20, nbdev: float = 2.0)
```

Boll

In [10]:
save_doc("mintalib.core", func_names = str.islower, strict=True)

# mintalib.core module

## calc_adx function

```python
calc_adx(prices, period=14, *, wrap: bool = False)
```

Average Directional Index

Args:
- period (int) : time period, default 14

## calc_atr function

```python
calc_atr(prices, period=14, *, wrap: bool = False)
```

Average True Range

Args:
- period (int) : time period, default 14    

## calc_avgprice function

```python
calc_avgprice(prices, *, wrap: bool = False)
```

Average Price

Value of (open + high + low + close) / 4

## calc_bbands function

```python
calc_bbands(prices, period=20, nbdev=2.0, *, wrap: bool = False)
```

Bollinger Bands

Args:
- period (int) : time period, default 20
- nbdev (float) : bands width in number of standard deviations

## calc_bop function

```python
calc_bop(prices, period=20, *, wrap: bool = False)
```

Balance of Power

Args:
- period (int) : time period, default 20

## calc_cci function

```python
calc_cci(prices, period=20, *, wrap: bool = False)
```

Commodity Channel Index

Args:
- p